## Data Preprocessing

There are two datasets in this projects. First one contains relative score differential and total score of each game of each team scrapped by me from ESPN. The other dataset is the "Advanced Team Statistics" of each team againg scrapped by me from Fox Sport on each day of the 2018-2019 season. You can go over databases, we have shared them. The statistics datasets have almost every predictor we want to train our model except the Home/Away information which we will calculate using schedule/score dataset and add it to our training dataset.

Although we have data of the whole season the model we want to build is going to use only the previous data to predict a game. For example if we want to predict 44th game of the season for Timberwolves the model will only use previous games to train and will predict the result of 44th game. Besides, model will not use the statistics of the team that we want to predict its game but its opponents'. What I mean is we will use previous opponents statistics as explanatory variables and the score of the game against that opponent as response variable and to predict 44th game of the season for Timberwolves we will give the statistics of the opponent to the model to predict the result of that game. 

In [1]:
import numpy as np
import sqlite3
import pandas as pd

In [2]:
# Connect to databases and fetch all 
con_schedule = sqlite3.connect("schedule_scores.db")
cursor_schedule = con_schedule.cursor()

con_stats = sqlite3.connect("team_stats.db")
cursor_stats = con_stats.cursor()

# List of 30 teams and city name dictionary to match the names used by ESPN and FoxSport
teams = ["CHA","PHI","TOR","BOS","CLE","IND","WSH","MIL","MIA","DET","NY","CHI","ORL","BKN","ATL","HOU","GS","POR","NO","MIN","SA","OKC","DEN","LAC","UTA","LAL","SAC","DAL","PHX","MEM"]
city_dic = {"atl":"Atlanta","bkn": "Brooklyn" ,"bos": "Boston", "cha":"Charlotte", "chi":"Chicago", "cle": "Cleveland", "dal": "Dallas", "den": "Denver", "det":"Detroit","gs":"Golden State", "hou": "Houston", "ind":"Indiana","lac":"LA", "lal": "Los Angeles","mem": "Memphis","mia":"Miami","mil":"Milwaukee","min":"Minnesota","no":"New Orleans","ny":"New York","okc":"Oklahoma City","orl":"Orlando","phi":"Philadelphia","phx":"Phoenix","por":"Portland","sa":"San Antonio","sac":"Sacramento","tor":"Toronto","uta":"Utah","wsh":"Washington" }

# Fetching the schedules of each team
schedules = {}
for team in teams:
    cursor_schedule.execute("SELECT * FROM {}".format(team.lower()))
    schedules[team.lower()] = cursor_schedule.fetchall()

# Fetching the stats of each team
stats = {}
for team in teams:
    cursor_stats.execute("SELECT * FROM {}".format(team.lower()))
    stats[team.lower()] = cursor_stats.fetchall()  
    


In [3]:
# Now we will write a function that creates x and y matrices to predict specific game
# To do this we are going add statistics of the opponents to schedule/score dataset 
 
def get_dataset(team,schedules,stats):
    
    schedule = schedules[team]
    
    schedule_df = pd.DataFrame(schedule, columns=['Game', 'Date', 'Opponent','Home/Away(1/0)','Score','TotalScore'])
    # Date column should be formatted from 'Oct 19 2019' to pandas date
    schedule_df['Date'] = pd.to_datetime(schedule_df['Date'],format='%b %d %Y')
    
    # We are going to add each opponents statisctis to that dataframe and than we are going to combine schedule_df and stats_df
    stats_df = pd.DataFrame(columns=["Date","GamesPlayed","OffRtg","DefRtg","Pace","FtRate","ThreeFgTend",
    "TrueS","Efg","TurnOver","OffReb","FtFga","EfgAllow","TurnOvAllow","DefRebAllow","FtFgaAllow"])
    
    # To fill stats_df we are goint to itterate over the rows of schedule_df.
    for index, row in schedule_df.iterrows():
        
        # team_stats.db has long city names instead of short ones like in the schedule_scores.db 
        # therefore a dictionary is used to match the team names ex. opponent_ = "Atlanta" -> opponent = "atl"
        opponent_ = row["Opponent"]
        opponent = list(filter(lambda x: x[1] == opponent_,list(city_dic.items())))[0][0]

        opponent_stats_table = stats[opponent]
        opponent_stats_df = pd.DataFrame(opponent_stats_table,columns=["Date","GamesPlayed","OffRtg","DefRtg","Pace","FtRate","ThreeFgTend",
    "TrueS","Efg","TurnOver","OffReb","FtFga","EfgAllow","TurnOvAllow","DefRebAllow","FtFgaAllow"])
        opponent_stats_df['Date'] = pd.to_datetime(opponent_stats_df['Date'],format='%b %d %Y')

        # We have a date from schedule and we basicaly try to find the statistics row that have the most 
        # similar date with schedule because the not every day's statistics are gathered
        date = row["Date"]
        stats_at_date = opponent_stats_df.iloc[opponent_stats_df.Date.searchsorted(date-pd.DateOffset(days=1))].to_frame().T
        stats_df = pd.concat([stats_df,stats_at_date])
        
    
    df = pd.concat([schedule_df.reset_index(drop=True),stats_df.reset_index(drop=True)], axis=1)
    # Changing the position of Home/Away column to make easier to chose x and y matrices
    homeAway_column = df.pop('Home/Away(1/0)')
    df.insert(7, 'Home/Away(1/0)', homeAway_column)

    return df

df = get_dataset("atl",schedules,stats)
df.head()

,Game,Date,Opponent,Score,TotalScore,Date,GamesPlayed,Home/Away(1/0),OffRtg,DefRtg,...,ThreeFgTend,TrueS,Efg,TurnOver,OffReb,FtFga,EfgAllow,TurnOvAllow,DefRebAllow,FtFgaAllow
0,1,2018-10-17,New York,-19.0,233.0,2018-10-31 00:00:00,7,0.0,106.5,110.5,...,33.7,0.522,0.489,11.7,24.2,16.2,0.544,14.5,75.8,21.3
1,2,2018-10-19,Memphis,-14.0,248.0,2018-11-02 00:00:00,7,0.0,105.5,101.8,...,35.0,0.557,0.512,12.4,17.4,25.7,0.524,16.4,82.6,20.2
2,3,2018-10-21,Cleveland,22.0,244.0,2018-11-01 00:00:00,8,0.0,109.8,117.7,...,25.1,0.538,0.489,12.7,29.6,22.7,0.585,13.6,70.4,19.9
3,4,2018-10-24,Dallas,7.0,215.0,2018-10-31 00:00:00,7,1.0,107.8,113.5,...,40.7,0.542,0.507,13.2,23.9,19.4,0.568,14.2,76.1,23.4
4,5,2018-10-27,Chicago,-12.0,182.0,2018-10-31 00:00:00,7,1.0,108.7,118.2,...,34.5,0.567,0.535,12.9,17.5,20.1,0.539,10.9,82.5,22.3


In [4]:
total_correct = 0
final_dev = 0
for team in teams:
    team = team.lower()
    correct_winner = 0
    total_deviation = 0
    for game_number in range(21,83):
        df = get_dataset(team,schedules,stats)

        x_train = df.iloc[:game_number-1,7:]
        y_train = df.iloc[:game_number-1,3]
        x_test = df.iloc[game_number-1,7:]
        y_test = df.iloc[game_number-1,3]

        # Multiple Linear Regression
        from sklearn.linear_model import LinearRegression
        regressor = LinearRegression()
        regressor.fit(x_train.values, y_train.values)
        y_pred = regressor.predict(np.array([x_test]))

        if y_pred*y_test > 0:
            correct_winner += 1
        total_deviation += abs(y_pred-y_test)
    total_correct += correct_winner
    final_dev += total_deviation

    print("{}: WinnerCorrect -> {} and PointDeviation -> {}".format(team.upper(),correct_winner/62,total_deviation/62))
print("---------------------")          
print("Overall: WinnerCorrect -> {} and PointDeviation -> {}".format(total_correct/62/30,final_dev/62/30))


CHA: WinnerCorrect -> 0.7096774193548387 and PointDeviation -> [12.07610887]
PHI: WinnerCorrect -> 0.5645161290322581 and PointDeviation -> [11.44606855]
TOR: WinnerCorrect -> 0.6129032258064516 and PointDeviation -> [11.88539567]
BOS: WinnerCorrect -> 0.6612903225806451 and PointDeviation -> [13.56215348]
CLE: WinnerCorrect -> 0.6774193548387096 and PointDeviation -> [11.55109627]
IND: WinnerCorrect -> 0.6774193548387096 and PointDeviation -> [12.11688823]
WSH: WinnerCorrect -> 0.6451612903225806 and PointDeviation -> [10.66948085]
MIL: WinnerCorrect -> 0.6451612903225806 and PointDeviation -> [12.21089812]
MIA: WinnerCorrect -> 0.5967741935483871 and PointDeviation -> [12.76266381]
DET: WinnerCorrect -> 0.5967741935483871 and PointDeviation -> [11.71068548]
NY: WinnerCorrect -> 0.7096774193548387 and PointDeviation -> [11.59702621]
CHI: WinnerCorrect -> 0.6290322580645161 and PointDeviation -> [13.49193548]
ORL: WinnerCorrect -> 0.3870967741935484 and PointDeviation -> [16.96377563]
